In [141]:
def read_packet_data(filename):

    with open(filename) as infile:
        raw_data = [line.strip() for line in infile]

    raw_data.append("")

    packet_pairs = list()
    for i in range(0, len(raw_data)):
        if len(raw_data[i]) == 0:
            packet_pairs.append([eval(raw_data[i - 2]), eval(raw_data[i - 1])])

    return packet_pairs


In [142]:
def compare_packets(packet_pair, undecided=True, right_order=None, verbose=False):

    left = packet_pair[0]
    right = packet_pair[1]

    if verbose:
        print(f" - Compare {left} vs {right}")

    if type(left) == int and type(right) == int:
        if left < right:
            right_order = True
            undecided = False
            if verbose:
                print(f" - Left side is smaller, so inputs are in the right order")
        elif left > right:
            right_order = False
            undecided = False
            if verbose:
                print(f" - Right side is smaller, so inputs are not in the right order")
    else:
        while undecided and (left or type(left) == int):
            if (type(left) == int and type(right) == list) or (
                type(left) == list and type(right) == int
            ):
                if type(left) == int:
                    left = [left]
                    if verbose:
                        print(
                            f" - Mixed types; convert left to {left} and retry comparison"
                        )
                else:
                    right = [right]
                    if verbose:
                        print(
                            f" - Mixed types; convert right to {right} and retry comparison"
                        )

            if len(left) > 0 and len(right) == 0:
                undecided = False
                right_order = False
                if verbose:
                    print(
                        "- Right side ran out of items, so inputs are not in the right order"
                    )
                break

            undecided, right_order = compare_packets(
                [left.pop(0), right.pop(0)], undecided, right_order, verbose
            )
            if not undecided:
                break

        if (
            undecided
            and (type(left) == list and len(left) == 0)
            and (type(right) == int or len(right) > 0)
        ):
            undecided = False
            right_order = True
            if verbose:
                print("- Left side ran out of items, so inputs are in the right order")

    return undecided, right_order


def compare_all(packet_pair_inputs, debug=False):
    sum_indices = 0
    print(f"{len(packet_pair_inputs)} packets")
    i = 0
    outcomes = list()
    undecideds = list()
    indices_sum = list()
    for packets_in in packet_pair_inputs:
        i += 1
        if debug:
            print(f"\n== Pair {i} ==")
            print(f"   - Comparing: [{packets_in[0]}, {packets_in[1]}]")
        undec, right_order = compare_packets(packets_in)
        outcomes.append(right_order)
        if debug:
            print(f"   - Right order: {right_order}")
        undecideds.append(undec)
        if right_order:
            sum_indices += i
            indices_sum.append(i)

    print(f"\nProcessed: {len(outcomes)}")
    print(f"Any undecideds: {any(undecideds)}")
    print(f"sum_indices: {sum_indices}")

    return sum_indices


packet_pairs = read_packet_data("inputs/day13.txt")

compare_all(packet_pairs, False)


150 packets

Processed: 150
Any undecideds: False
sum_indices: 5808


5808

--- Part Two ---



In [143]:
import copy


def read_packet_data_part_two(filename):
    with open(filename) as infile:
        raw_data = [eval(line.strip()) for line in infile if len(line.strip()) > 0]
    return raw_data + [[[2]], [[6]]]


def sort_list(packet_data):
    i = 1
    while i < len(packet_data):
        j = i
        while (
            j > 0
            and not compare_packets(
                [copy.deepcopy(packet_data[j - 1]), copy.deepcopy(packet_data[j])]
            )[1]
        ):
            packet_data[j - 1], packet_data[j] = packet_data[j], packet_data[j - 1]
            j -= 1

        i += 1

    print(f"decoder key: {(packet_data.index([[2]])+1)*(packet_data.index([[6]])+1)}")

    return packet_data


packets = read_packet_data_part_two("inputs/day13.txt")
p = sort_list(packets)


decoder key: 22713
